<a href="https://colab.research.google.com/github/baker371/p-threat-model-graph-visualisation/blob/main/visjs/Treat_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install pyvis

In [4]:
#@title

import numpy as np
import pandas as pd
from pyvis.network import Network
import json


id = "1BouPWCBx9rD8ORVC3GoStxGleOwDr9vg-uql-_DG66M"

trust__boundaries = pd.read_csv("https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet=trust__boundaries".format(id))

assets = pd.read_csv("https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet=assets".format(id))

security_controls = pd.read_csv("https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet=security_controls".format(id))

threat_actors = pd.read_csv("https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet=threat_actors".format(id))

threat_table = pd.read_csv("https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet=threat_table".format(id))

# Combining Sheet 1 and Sheet 2, because they both have the Asset/Trust Boundary ID appearing in the first column of sheet 5, renaming Asset ID to Trust Boundary ID

comb12 =  pd.concat([trust__boundaries,assets.rename(columns={'Asset ID':'Trust Boundary ID'})], ignore_index=True)

# Adding Descriptions against the Asset/Trust Boundary ID	code value in the combined df comb12

new_col = threat_table['Asset/Trust Boundary ID'].map(comb12.set_index('Trust Boundary ID')['Description'])

threat_table.insert(loc = 1, column = 'Boundary Description', value = new_col) 

# Adding Descriptions against the Security Control ID	code value in security_controls

threat_table['Security Control Description'] = threat_table['Security Control ID'].map(security_controls.set_index('Security Control ID')['Description'])

# Adding Descriptions and skills against the Threat Actor ID	code value in threat_actors

new_colm = threat_table['Threat Actor ID'].map(threat_actors.set_index('Threat Actor ID')['Description'])
new_coln = threat_table['Threat Actor ID'].map(threat_actors.set_index('Threat Actor ID')['Skills'])
threat_table.insert(loc = 4, column = 'Threat Actor Description', value = new_colm) 
threat_table.insert(loc = 5, column = 'Skills', value = new_coln) 

threat_table.fillna(0,inplace=True)


threat_table

,Asset/Trust Boundary ID,Boundary Description,Threat,Threat Actor ID,Threat Actor Description,Skills,Vulnerability,Risk,Risk Possibility,Risk Impact,Risk Level,Security Control ID,Security Control Description
0,TB01,ICAP Client - ICAP Server,External attacher spoofing and sending packets...,TA04,External Attacker,Intermediate,Open port Cloud ICAP Server,Service unavailability and unplanned costs,Likely,Significant,Med Hi,C03,Conditional Access Policy
1,TB02,SIEM- Cloud Auditing System,Attacker compromising the availability and int...,TA02,External Attacher with SIEM access or Maliciou...,Intermediate,Log storage and transit is read only 0-day vul...,"Unplanned costs, financial and reputational da...",Very Unlikely,Moderate,Low Med,Accept,0
2,A07,Rebuild Store,Outdated cache affecting file outcome,TA03,Malicious File,Beginner,Files are not Processed using latest version o...,Reduced service availability and unplanned and...,Likely,Significant,Med Hi,C04,Clear cache before library upgrade
3,A06,Audit Store,Attacker accessing client sensitive data,TA01,External Attacher with access to Cloud Platform,Advanced,No encryption in transit and non-anoymised data,Reputational damage and unplanned costs due to...,Unlikely,Minor,Low Med,Accept,0
4,TB01,ICAP Client - ICAP Server,External attacker doing Session Hijacking/ MITM,TA05,External Attacker,Advanced,No authetication / encryption of ICAP traffic,Reputational damage and unplanned costs due to...,Unlikely,Significant,Medium,C03,Conditional Access Policy
5,TB01,ICAP Client - ICAP Server,Tampering,TA05,External Attacker,Advanced,No siging ability/validation,Input/Output Tampered,Unlikely,Minor,Low,C03,Conditional Access Policy
6,A08,Content Store,Client data being exposed to an attacker in th...,TA01,External Attacher with access to Cloud Platform,Advanced,Sensitive data storage in the cloud,Reputational damage and unplanned costs due to...,Very Unlikely,Significant,Medium,C05,Dissalow soft Delete
7,A09,Rebuild,Malicious file parsing through Rebuild,TA06,Malicious File Sender,Advanced,Buffer Overflow Vulnerability,Unplanned costs due to GW Egine repair after b...,Unlikely,Significant,Medium,C01,Rebuild in Separate Docker Container
8,A09,Rebuild,Malicious file parsing through Rebuild,TA06,Malicious File Sender,Advanced,DoD Vulnerability,Service unavailability - Minor due to docker c...,Unlikely,Minor,Low,C01,Rebuild in Separate Docker Container
9,A10,Analyse,Malicious file parsing through Analyse,TA06,Malicious File Sender,Advanced,DoD Vulnerability,Service unavailability - Minor due to docker c...,Unlikely,Minor,Low,C01,Rebuild in Separate Docker Container


In [5]:
#@title

# data = threat_table.to_json(orient ='records') 

# data

#https://www.freeformatter.com/json-formatter.html

#Export to JSON

threat_table.to_json(r'threat_table.json')

In [6]:
G = Network(height="1500", width="70%", bgcolor="#222222", font_color="white", directed = True)

# set the physics layout of the network
G.barnes_hut()

Boundary_description       = threat_table['Boundary Description']
Threat                     = threat_table['Threat']
Threat_actor_description   = threat_table['Threat Actor Description']
Skills                     = threat_table['Skills']
Vulnerability              = threat_table['Vulnerability']
Risk                       = threat_table['Risk']
Risk_possibility           = threat_table['Risk Possibility']
Risk_impact                = threat_table['Risk Impact']
Risk_level                 = threat_table['Risk Level']
Security_description       = threat_table['Security Control Description']

columns = zip(Boundary_description, Threat, Threat_actor_description,Skills,Vulnerability ,Risk,Risk_possibility,Risk_impact,Risk_level,Security_description)

for column in columns:
    boundary = column[0]
    threat = column[1]
    actor_description = column[2]
    skills = column[3]
    vulnerability = column[4]
    risk = column[5]
    r_posibility = column[6]
    r_impact = column[7]
    r_level = column[8]
    security_description = column[9]
    
    G.add_node(boundary, color = '#03DAC6', shape='box')
    G.add_node(threat, color = '#03DAC6', shape='box')
    G.add_node(actor_description, color = '#F39C12', shape='box')
    G.add_node(skills, color = '#EC407A', shape='circle')
    G.add_node(vulnerability, color = '#27AE60', shape='box')
    G.add_node(risk, color = '#F44336', shape='box')
    G.add_node(r_impact, color = '#CCFF00', shape='circle')
    G.add_node(security_description, color = '#3333FF', shape='box')

    G.add_edge(boundary, threat,color='#990099')
    G.add_edge(threat,actor_description)
    G.add_edge(actor_description,vulnerability,color='#009966')
    G.add_edge(actor_description,skills,color='#6699FF')
    G.add_edge(skills,vulnerability,color='#ffffff')
    G.add_edge(vulnerability,risk,value=r_posibility,color='#546E7A')
    G.add_edge(risk,r_impact,color='#CCFF33')
    G.add_edge(r_impact,security_description,value=r_level,color='#F8C471')

G.show_buttons(filter_=['edges','physics'])
G.show("threat_model.html") 


In [ ]:
#import IPython
#IPython.display.HTML(filename='threat_model.html')